In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd

import collections
import re
import pprint as pp
import numpy as np
import collections

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

import math
import gzip
import pickle as pkl
from datetime import datetime
import matplotlib.pyplot as plt

import joblib

import fonctions
import itertools
from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join
from collections import defaultdict

import random
random.seed(0)

Using TensorFlow backend.


In [2]:
def get_behavior(directory,doc_name):
    
    all_features = ["text_tokens", "hashtags", "tweet_id", 
                    "present_media", "present_links", 
                    "present_domains", "tweet_type","language", 
                    "tweet_timestamp", "engaged_with_user_id",
                    "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                    "engaged_with_user_is_verified", "engaged_with_user_account_creation",
                    "engaging_user_id", "engaging_user_follower_count", 
                    "engaging_user_following_count", "engaging_user_is_verified",
                    "engaging_user_account_creation", "engagee_follows_engager"]
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep='\x01', header=None)

    labels = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
    all_variables = all_features + labels
    df.columns = all_variables
        
    df['reply_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['reply_timestamp'] ]
    df['retweet_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_timestamp'] ]
    df['retweet_with_comment_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_with_comment_timestamp'] ]
    df['like_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['like_timestamp'] ]
        
    df['link']=[1 if type(x) is str else 0 for x in df['present_links']]
    df['photo'] = [ 1 if type(x) is str and x.count('Photo')>0 else 0 for x in df['present_media'] ]
    df['video'] = [ 1 if type(x) is str and x.count('Video')>0 else 0 for x in df['present_media'] ]
    df['photo_video']= [1 if x1+x2>0 else 0 for x1,x2 in zip(df.photo,df.video) ]
    df['gif'] = [ 1 if type(x) is str and x.count('GIF')>0 else 0 for x in df['present_media'] ]
    df['text'] = [1 if x1+x2+x3+x4==0 else 0 for x1,x2,x3,x4 in zip(df.link,df.photo,df.video,df.gif) ]
    df = df.filter(labels+['text','link','photo_video','gif','engaging_user_id'],axis=1)

    return df

def multimedia_on_chunk(directory, chunk, chunk_id):

    multimedias = {}
    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']

    iteration=1
    for batch_file in chunk:
        
        df = get_behavior(directory, batch_file)
        
        select = { k:[0,0,0,0]  for k in np.unique(df.engaging_user_id) if k not in multimedias.keys()  }
        multimedias.update(select)
        
        dictio = collections.defaultdict(list)
        for idx, engagement in enumerate(engagements):
            buff =  df[ df[engagement]==1 ]
            buff = buff.groupby(['engaging_user_id']).sum()
            dictio = { k: [text, link, photo_video, gif]  for k, text, link, photo_video, gif in zip(buff.index, buff.text, buff.link, buff.photo_video, buff.gif)}
            { update_multimedia(multimedias, k, v) for k,v in dictio.items() }
        
        print(iteration)
        iteration=iteration+1
    
    print('saving...')

    with gzip.open('/home/maxime/Desktop/RecSys2020/trends/taste_content_user_{}.pkl.gz'.format(chunk_id), 'wb') as f:
        pkl.dump(multimedias,f)

    return True

def update_multimedia(multimedias, k, v):
    multimedias[k] = np.add(multimedias[k], v)


In [5]:
%%time

batch_path = '/home/maxime/Desktop/RecSys2020/data/batches'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 12)

directory = '/home/maxime/Desktop/RecSys2020/data/batches/'
if __name__ == '__main__':
    
    # Setup a list of processes that we want to run
    processes = [ mp.Process(target=multimedia_on_chunk, args=(directory, chunk, chunk_id) ) for chunk_id, chunk in zip([9,10,11],[ chunks[9],chunks[10],chunks[11] ]) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 

1
1
1
2
2
2
3
3
3
4
4
4
5
5
5
6
6
6
7
7
7
8
8
9
8
9
9
10
10
11
10
11
12
11
12
12
13
13
14
13
14
14
15
15
15
16
16
16
17
17
17
18
18
19
18
19
20
19
20
21
20
21
22
21
22
23
22
23
24
23
24
25
24
25
26
25
26
27
26
27
28
27
28
29
28
29
30
29
30
31
30
31
32
31
32
33
32
33
34
33
35
34
34
36
35
35
37
36
36
38
37
38
37
39
39
38
40
40
39
41
40
41
42
41
42
43
42
43
44
43
44
45
45
44
46
45
46
47
46
48
47
49
47
48
50
48
49
49
51
50
50
51
52
51
52
53
52
54
53
53
54
55
55
54
56
56
55
57
57
56
58
58
57
59
58
59
60
60
59
61
61
60
62
62
63
61
63
62
64
64
63
65
65
64
66
66
67
65
67
68
68
66
69
69
67
70
70
68
71
71
69
72
72
70
73
73
71
74
72
74
73
75
75
76
74
76
77
75
77
78
76
78
79
77
79
80
78
81
80
79
81
80
82
81
83
82
84
83
82
85
84
83
86
85
84
87
86
85
88
87
86
89
88
87
90
89
88
90
91
89
91
92
90
93
91
92
94
93
92
95
94
93
96
95
94
97
96
95
98
97
96
99
98
97
99
100
98
99
100
101
100
101
102
101
103
102
102
104
103
103
105
104
104
106
105
105
107
106
106
108
107
107
109
108
110
108
109
109
111
110
110


In [6]:
import metric_loading
taste_content = metric_loading.get_user_content_taste()
taste_content = {k : np.divide(v,[sum(v)]*4) for k,v in tqdm(taste_content.items()) if sum(v)>5 }
with gzip.open('/home/maxime/Desktop/RecSys2020/trends/taste_content_5.pkl.gz','wb') as f:
    pkl.dump(taste_content,f)

present_user_content_taste
0
1
2
3
4
5
6
7
8
9
10
11
global_taste_content_user formating
